# Финальная версия блокнота обучения модели предсказания цен
Тут будет сводный код результатов экспериментов всех прошлых блокнотов. Только лучшие показатели.
Лучшие показатели прошлого эксперимента:
- R2 Score: 0.9261
- MSE:      515670500.4637
- RMSE:     22708.3795
- MAE:      13696.3981
- MSLE:     0.0141
- RMSLE:    0.1187
- MAPE:     0.0817

Что соответствует kaggle submission score = 12877 и 114 месту в leaderboard из 7060 участников.
Тут попробуем финальные эксперименты с:
- еще раз логарифмируем целевую переменную
- попробуем заменить train_test_split на KFold (мы использовали train_test_split. Это значит, что мы обучались на 70% данных и валидировались только на одной, фиксированной 30%-й выборке. Такая валидация очень нестабильна и ненадёжна. Результаты сильно зависят от того, какие именно данные попали в эти 30%. Улучшения на этой выборке могут не переноситься на ЛБ. Нужно попробовать использовать кросс-валидацию (Cross-Validation, CV), например, KFold.)
- попробуем логарифмировать признаки вместо winsoring-а
- попробуем еще парочку новых признаков создать
- тюнинг гиперпараметров после внедрения CV

In [2]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import mlflow
import mlflow.sklearn
import pandas as pd
import warnings

from catboost import CatBoostRegressor
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from utils.data_manager import DataManager
from utils.model_manager import ModelManager

In [3]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [4]:
dm = DataManager()
mm = ModelManager()

# Отключаем автологгирование, чтобы использовать ручное
mlflow.sklearn.autolog(disable=True)
warnings.filterwarnings("ignore", module="mlflow") # Игнорируем предупреждения MLflow


In [5]:
RANDOM_STATE = 42

## 1. Загрузка данных

In [6]:
data_path = 'data/home-data-for-ml-course'
train_data = pd.read_csv(data_path + '/train.csv')
test_data = pd.read_csv(data_path + '/test.csv')

## 2. Предобработка данных 

In [7]:
# Определение колонок для удаления
intuitively_bad_features = [
    'LotShape',  # Общая форма участка
    'LandContour',  # Рельеф участка
    'LotConfig',  # Конфигурация участка
    'LandSlope',  # Уклон участка
    'MiscFeature',
    'MiscVal',
]
bad_columns = dm.get_all_nan_cols(train_data)
bad_columns.append('Id')
bad_columns.extend(intuitively_bad_features)

In [8]:
# Разделение на X / y
X, y = dm.split_data_set_to_x_y(train_data, 'SalePrice')
print(X.shape, y.shape)
X_test = test_data.copy()
print(X_test.shape)

(1460, 80) (1460,)
(1459, 80)


In [9]:
X.drop(columns=bad_columns, inplace=True)
X_test.drop(columns=bad_columns, inplace=True)

In [10]:
def make_feature_eng_great_again(train_X_in, test_X_in):
    """Хелпер, который создает фичи, логарифмирует и выравнивает колонки."""
    # Работаем с копиями, чтобы не изменять оригинальные X, X_test вне функции
    train_X = train_X_in.copy()
    test_X = test_X_in.copy()

    # Словарь качественных признаков
    quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}

    def create_features(df):
        # Interactions (с проверкой на наличие колонок)
        if 'Neighborhood' in df.columns and 'MSZoning' in df.columns:
            df['Neighborhood_Zoning'] = df['Neighborhood'].astype(str) + '_' + df['MSZoning'].astype(str)
        if 'SaleType' in df.columns and 'SaleCondition' in df.columns:
            df['SaleType_Condition'] = df['SaleType'].astype(str) + '_' + df['SaleCondition'].astype(str)

        # Quality Score
        df['TotalQualScore'] = 0
        quality_cols = ['ExterQual', 'KitchenQual', 'BsmtQual', 'HeatingQC', 'GarageQual', 'FireplaceQu']
        for col in quality_cols:
            if col in df.columns:
                 df['TotalQualScore'] += df[col].map(quality_dict).fillna(0)

        # Porch/Deck Area and Flags
        df['PorchDeckArea'] = 0
        porch_cols = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
        for col in porch_cols:
             if col in df.columns:
                df['PorchDeckArea'] += df[col].fillna(0)

        if 'Fireplaces' in df.columns:
            df['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)
        if 'GarageType' in df.columns:
            df['HasGarage'] = (~df['GarageType'].isna()).astype(int)
        if 'Fence' in df.columns:
            df['HasFence'] = (~df['Fence'].isna()).astype(int)
        df['HasPorchDeck'] = (df['PorchDeckArea'] > 0).astype(int)

        return df

    def log_features(df, cols_to_log_list):  # Принимает СПИСОК колонок
        print(f"Applying log1p to: {cols_to_log_list}")
        for col_name in cols_to_log_list:
            if col_name in df.columns:
                # Добавим проверку на отрицательные значения перед логарифмированием
                if (df[col_name] < 0).any():
                     print(f"Warning: Column {col_name} contains negative values. Skipping log1p.")
                else:
                    df[col_name] = np.log1p(df[col_name])
            else:
                print(f"Warning: Column {col_name} not found in DF during log transform.")
        return df

    # 1. Создаем фичи
    train_X = create_features(train_X)
    test_X = create_features(test_X)
    print("Features created.")

    # 2. Определяем колонки для логарифмирования (ТОЛЬКО по трейну)
    numeric_cols = train_X.select_dtypes(include=np.number).columns
    skew_values = train_X[numeric_cols].skew()
    # Используем .index.tolist() чтобы получить список имен
    cols_to_log_list = skew_values[skew_values > 1].index.tolist()
    print(f"Columns identified for logging: {cols_to_log_list}")

    # 3. Логарифмируем (используя ОДИН и тот же список)
    # train_X = log_features(train_X, cols_to_log_list)
    # test_X = log_features(test_X, cols_to_log_list)
    # print("Log transform applied.")

    # 4. Согласуем и сортируем колонки ПОСЛЕ всех манипуляций
    final_feature_cols = sorted(train_X.columns.tolist()) # Сортируем для стабильности
    train_X = train_X[final_feature_cols]
    test_X = test_X.reindex(columns=final_feature_cols, fill_value=0)
    print("Columns aligned and sorted.")

    return train_X, test_X

In [11]:
# Вызываем функцию с правильными данными (X, X_test)
X, X_test = make_feature_eng_great_again(X, X_test)

print("\nProcessing complete. Final shapes:")
print(f"X_processed: {X.shape}")
print(f"X_test_processed: {X_test.shape}")
print("\nExample processed X:")
print(X.head())

Features created.
Columns identified for logging: ['MSSubClass', 'LotFrontage', 'LotArea', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'TotalBsmtSF', '1stFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtHalfBath', 'KitchenAbvGr', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'PorchDeckArea', 'HasFence']
Columns aligned and sorted.

Processing complete. Final shapes:
X_processed: (1460, 81)
X_test_processed: (1459, 81)

Example processed X:
   1stFlrSF  2ndFlrSF  3SsnPorch Alley  BedroomAbvGr BldgType BsmtCond  \
0       856       854          0   NaN             3     1Fam       TA   
1      1262         0          0   NaN             3     1Fam       TA   
2       920       866          0   NaN             3     1Fam       TA   
3       961       756          0   NaN             3     1Fam       Gd   
4      1145      1053          0   NaN             4     1Fam       TA   

  BsmtExposure  BsmtFinSF1  BsmtFinSF2  ... ScreenPorch Street  TotRmsAbvGrd  \


In [12]:
# Получение числовых колонок
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
# Получение нечисловых колонок (всех остальных)
non_numeric_columns = X.select_dtypes(exclude=['float64', 'int64']).columns

## 3. Обучаем модель с CV и корректируя данные

In [13]:
# --- Логарифмирование целевой переменной ---
y_log = np.log1p(y)

#### Нормализация данных через ColumnTransformer и Pipeline
В данном кейсе мы реализуем Заполнение числовых пропусков с помощью модели (Predictive imputation). Т.е. то, что пропущено в числовых признаках - будем заполнять не медианой или средним, а будем обучать модель, которая будет предсказывать пропуски (IterativeImputer + RandomForestRegressor)

In [14]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# Пайплайн для числовых признаков (итеративное заполнение)
numeric_transformer = Pipeline(steps=[
    # ('imputer', IterativeImputer(
    #     estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
    #     max_iter=10,
    #     random_state=RANDOM_STATE
    # )),
    ('imputer', SimpleImputer(strategy='mean')),
    # ('scaler', StandardScaler())
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# --- Объединяем препроцессоры ---
preprocessor = ColumnTransformer(
    transformers=[
        # Применяем к исходным числовым колонкам
        ('num', numeric_transformer, numeric_columns),
        # Применяем к исходным категориальным колонкам
        ('cat', categorical_transformer, non_numeric_columns)
    ],
    remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
    verbose_feature_names_out=False  # Чтобы имена колонок не менялись на 'num__colname' и т.д.
)

default_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 100  # ?
}
catboost_params = {
    'iterations': 3000,  # Увеличиваем запас итераций, учитывая early_stopping
    'learning_rate': 0.03,
    'depth': 6,
    'loss_function': 'RMSE', # Оптимизируем RMSE на логарифмированной шкале
    'verbose': 0,
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 100 # Включаем early stopping
}
optuna_best_params = {'iterations': 2279,
 'depth': 5,
 'learning_rate': 0.01489174151229139,
 'l2_leaf_reg': 2.3495954457881623,
 'bagging_temperature': 0.9217322652060611,
 'loss_function': 'RMSE',
 'random_seed': 42,
 'verbose': 0}
catboost_params = optuna_best_params

# Включить Early Stopping очень рекомендуется в параметрах. Это позволит модели на каждом фолде останавливаться тогда, когда метрика на валидационной части этого фолда (X_val, y_val_log) перестает улучшаться. Это самый надежный способ подобрать оптимальное число итераций для каждого фолда и избежать переобучения

# --- Финальный пайплайн ---
# Мы будем использовать preprocessor и модель отдельно в цикле CV
# для корректной работы early stopping с пайплайном sklearn. Поэтому нам финальный пайплайн - не нужен.
# final_pipeline = Pipeline([
#     ('preprocessing', preprocessor),
#     ('model', CatBoostRegressor(**catboost_params))
# ])

In [15]:
"""
Пояснения к тому, что тут происходит:
oof_predictions — это массив, который в итоге будет содержать предсказания для каждого объекта из исходного тренировочного набора (X). Важно, что предсказание для конкретного объекта (например, дома №100) делается той моделью (из цикла CV), которая обучалась без этого объекта. Надежная оценка качества: OOF-предикты позволяют посчитать метрику качества (например, oof_rmse) на всем тренировочном наборе, при этом каждое предсказание было сделано "честно" (модель не видела этот объект при обучении). 
Эта оценка часто бывает более надежной, чем простое усреднение метрик по фолдам (mean_cv_rmse), так как она считается на полном наборе данных один раз. 
Можно сравнить oof_predictions с реальными значениями y_log (или y), чтобы понять, на каких объектах модель ошибается сильнее всего.
Стэкинг/Блендинг: OOF-предикты часто используются как новые признаки для обучения модели второго уровня (мета-модели) в ансамблях (стэкинг).

Как генерируются данные для сабмишна:
1. Модель обучается на своей части тренировочных данных (X_train, y_train_log).
2. Эта обученная модель делает предсказания на всем тестовом наборе test_X.
3. Усреднение: Ключевая строка здесь: test_predictions_log += test_preds_fold_log / N_FOLDS. Она накапливает сумму предсказаний со всех фолдов, деленных на их количество. То есть, после завершения цикла test_predictions_log будет содержать среднее арифметическое предсказаний от всех N_FOLDS моделей, обученных на разных частях тренировочных данных. 

Усреднение vs. Обучение на всех данных: Обучить модель на всех данных (X, y_log) после цикла и сделать предсказание — это тоже валидный и часто используемый подход. Плюсы: модель использует максимум данных. Минусы: это требует дополнительного шага обучения; иногда ансамбль (усреднение предсказаний с фолдов) показывает себя немного лучше на тесте за счет большей стабильности и разнообразия моделей. Выбор между этими двумя подходами часто зависит от конкретной задачи и данных, стоит пробовать оба. 
"""

# --- Кросс-Валидация ---
N_FOLDS = 5  # Например, 5 или 10
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

oof_predictions_log = np.zeros(X.shape[0])  # Для хранения out-of-fold предсказаний
test_predictions_log = np.zeros(test_data.shape[0])  # Для хранения предсказаний на тесте
fold_rmses = []
fold_best_iterations = []  # Будем сохранять лучшие итерации

mlflow.set_experiment("Final Version With CV LogTransform EarlyStopping v4")
with mlflow.start_run() as run:  # Сохраняем run для логирования артефактов
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y_log)):
        # # Пример кода без early stopping round на pipeline
        # print(f"--- Fold {fold+1}/{N_FOLDS} ---")
        # X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        # y_train_log, y_val_log = y_log.iloc[train_idx], y_log.iloc[val_idx]
        # 
        # # Обучаем пайплайн на фолде
        # final_pipeline.fit(X_train, y_train_log)
        # 
        # # Предсказания на валидационном фолде (в лог. шкале)
        # val_preds_log = final_pipeline.predict(X_val)
        # oof_predictions[val_idx] = val_preds_log  # Сохраняем
        # 
        # # Предсказания на тестовом наборе (в лог. шкале)
        # # Усредняем предсказания по фолдам
        # test_preds_fold_log = final_pipeline.predict(X_test)
        # test_predictions_log += test_preds_fold_log / N_FOLDS
        # 
        # # Оценка на фолде (в исходной шкале)
        # val_preds_orig = np.expm1(val_preds_log)
        # y_val_orig = np.expm1(y_val_log)
        # fold_rmse = np.sqrt(mean_squared_error(y_val_orig, val_preds_orig))
        # print(f"Fold {fold+1} RMSE: {fold_rmse}")
        # fold_rmses.append(fold_rmse)
        
    # --- Итоговая оценка CV ---
        
        # Код для eval и early_stopping без исопльзования pipeline, только preprocessor
        print(f"--- Fold {fold+1}/{N_FOLDS} ---")
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train_log, y_val_log = y_log.iloc[train_idx], y_log.iloc[val_idx]

        # 1. Обучаем препроцессор ТОЛЬКО на трейне текущего фолда
        preprocessor.fit(X_train)

        # 2. Трансформируем трейн и валидацию
        X_train_prep = preprocessor.transform(X_train)
        X_val_prep = preprocessor.transform(X_val)
        # Трансформируем тест (препроцессор уже обучен)
        X_test_prep = preprocessor.transform(X_test)

        # 3. Обучаем модель с early stopping
        model = CatBoostRegressor(**catboost_params)
        model.fit(X_train_prep, y_train_log,
                  eval_set=[(X_val_prep, y_val_log)],
                  verbose=0  # Отключаем вывод внутри fit, т.к. early stopping сам выведет инфо
                 )

        # Сохраняем лучшую итерацию
        best_iter = model.get_best_iteration()
        fold_best_iterations.append(best_iter)
        print(f"Best iteration for fold {fold+1}: {best_iter}")

        # 4. Предсказания на валидации и тесте
        val_preds_log = model.predict(X_val_prep)
        test_preds_fold_log = model.predict(X_test_prep)

        # 5. Сохраняем OOF и усредняем тестовые предсказания
        oof_predictions_log[val_idx] = val_preds_log
        test_predictions_log += test_preds_fold_log / N_FOLDS

        # 6. Оценка на фолде (в исходной шкале)
        val_preds_orig = np.expm1(val_preds_log)
        y_val_orig = np.expm1(y_val_log)
        fold_rmse = np.sqrt(mean_squared_error(y_val_orig, val_preds_orig))
        print(f"Fold {fold+1} RMSE: {fold_rmse}")
        fold_rmses.append(fold_rmse)
        mlflow.log_metric(f"fold_{fold+1}_rmse", fold_rmse, step=fold+1)
        mlflow.log_metric(f"fold_{fold+1}_best_iter", best_iter, step=fold+1)
    
    # --- Итоговая оценка CV ---
    mean_cv_rmse = np.mean(fold_rmses)
    std_cv_rmse = np.std(fold_rmses)
    oof_rmse = np.sqrt(mean_squared_error(np.expm1(y_log), np.expm1(oof_predictions_log)))

    print(f"\nMean CV RMSE: {mean_cv_rmse:.4f} +/- {std_cv_rmse:.4f}")
    print(f"OOF RMSE: {oof_rmse:.4f}")
    print(f"Mean best iteration: {np.mean(fold_best_iterations):.0f}")

    # Логгирование итоговых метрик вручную
    mlflow.log_metric("mean_cv_rmse", mean_cv_rmse)
    mlflow.log_metric("std_cv_rmse", std_cv_rmse)
    mlflow.log_metric("oof_rmse", oof_rmse)
    mlflow.log_metric("mean_best_iteration", np.mean(fold_best_iterations))

--- Fold 1/5 ---
Best iteration for fold 1: 1752
Fold 1 RMSE: 25486.39651124402
--- Fold 2/5 ---
Best iteration for fold 2: 2277
Fold 2 RMSE: 22718.05542108188
--- Fold 3/5 ---
Best iteration for fold 3: 300
Fold 3 RMSE: 35592.894340633335
--- Fold 4/5 ---
Best iteration for fold 4: 2278
Fold 4 RMSE: 25127.851425298868
--- Fold 5/5 ---
Best iteration for fold 5: 2210
Fold 5 RMSE: 18302.147081106636

Mean CV RMSE: 25445.4690 +/- 5683.9886
OOF RMSE: 26072.5836
Mean best iteration: 1763


In [16]:
# К чему стремимся
# --- Fold 1/5 ---
# Best iteration for fold 1: 382
# Fold 1 RMSE: 26869.182804365337
# --- Fold 2/5 ---
# Best iteration for fold 2: 997
# Fold 2 RMSE: 24563.54129876178
# --- Fold 3/5 ---
# Best iteration for fold 3: 102
# Fold 3 RMSE: 34997.71799459465
# --- Fold 4/5 ---
# Best iteration for fold 4: 843
# Fold 4 RMSE: 26379.416273888422
# --- Fold 5/5 ---
# Best iteration for fold 5: 525
# Fold 5 RMSE: 19230.71124945864
# 
# Mean CV RMSE: 26408.1139 +/- 5077.6424
# OOF RMSE: 26891.8377
# Mean best iteration: 570


In [17]:
# --- Создание файла для сабмишна ---
final_test_predictions = np.expm1(test_predictions_log)

submission = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': final_test_predictions})
submission_filename = 'submission_final_clean_version_v9.csv'
submission.to_csv(submission_filename, index=False)
print(f"\nSubmission file '{submission_filename}' ready (example):")
print(submission.head())

# Логгирование артефакта в mlflow (если нужно и run еще активен)
# Если запускается в отдельной ячейке, нужно получить run_id
last_run_id = mlflow.last_active_run().info.run_id
with mlflow.start_run(run_id=last_run_id):
     mlflow.log_artifact(submission_filename)
# Или если код выполняется последовательно внутри одного with:
# try:
#     with mlflow.start_run(run_id=run.info.run_id): # Используем сохраненный run
#          mlflow.log_artifact(submission_filename)
#          print(f"Submission file logged to MLflow run {run.info.run_id}")
# except Exception as e:
#      print(f"Could not log submission artifact to MLflow: {e}")


Submission file 'submission_final_clean_version_v9.csv' ready (example):
     Id      SalePrice
0  1461  125692.060111
1  1462  155794.334872
2  1463  185180.945498
3  1464  193718.128009
4  1465  187230.687000


In [16]:
# --- Опционально: Финальное обучение на всех данных ---
# Можно попробовать сделать результаты еще и так
final_iterations = int(np.mean(fold_best_iterations)) # Берем среднее по фолдам
print(f"\nTraining final model on all data with {final_iterations} iterations...")
final_catboost_params = catboost_params.copy()
final_catboost_params['iterations'] = final_iterations
# Убираем early stopping для финального обучения
if 'early_stopping_rounds' in final_catboost_params:
    del final_catboost_params['early_stopping_rounds']

final_model_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', CatBoostRegressor(**final_catboost_params))
])

final_model_pipeline.fit(X, y_log)
print("Final model trained.")
# Эту модель можно сохранить и использовать для предсказаний
joblib.dump(final_model_pipeline, 'final_house_price_model.joblib')


Training final model on all data with 1004 iterations...
Final model trained.


['final_house_price_model.joblib']

In [17]:
# Предсказание на тесте финальной моделью (альтернатива усреднению)
final_test_pred_log_single_model = final_model_pipeline.predict(X_test)
final_test_pred_single_model = np.expm1(final_test_pred_log_single_model)
submission_single = pd.DataFrame({'Id': test_data['Id'], 'SalePrice': final_test_pred_single_model})
submission_single.to_csv('submission_final_single_model.csv', index=False)
print("\nSubmission file from single final model ready.")


Submission file from single final model ready.


# ИТОГО
Выше мы получили корректный код с K-FOLD валидацией, логарифмированием целевой переменной и предсказанием. Теперь его надо оптимизировать. Попробуем optuna.

In [16]:
import optuna

# Определяем KFold один раз вне objective
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

# --- Функция цели для Optuna ---
def objective(trial):
    """Функция, которую Optuna будет минимизировать."""

    # 1. Определяем подбираемые параметры CatBoost
    catboost_params = {
        'iterations': trial.suggest_int('iterations', 1000, 4000), # Даем запас для early stopping
        'depth': trial.suggest_int('depth', 4, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True), # Логарифмическая шкала часто лучше для LR
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1.0, 10.0, log=True), # Логарифмическая шкала для регуляризации
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0), # Параметр для контроля случайности бутстрэпа
        # Можно добавить другие параметры: 'colsample_bylevel', 'subsample', 'border_count' и т.д.
        'loss_function': 'RMSE',
        'verbose': 0, # Отключаем вывод CatBoost внутри objective
        'random_seed': RANDOM_STATE,
        'early_stopping_rounds': 100 # Фиксированное значение для early stopping
    }

    fold_rmses = [] # Список для хранения RMSE по фолдам для текущего trial

    # 2. Запускаем цикл KFold CV
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y_log)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train_log, y_val_log = y_log.iloc[train_idx], y_log.iloc[val_idx]

        try:
            # 3. Препроцессинг (fit на трейне, transform на трейне и валидации)
            # Важно: используем копию препроцессора или fit/transform в каждом фолде
            # Если preprocessor - это ColumnTransformer, он обучается быстро
            preprocessor.fit(X_train)
            X_train_prep = preprocessor.transform(X_train)
            X_val_prep = preprocessor.transform(X_val)

            # 4. Обучение модели CatBoost
            model = CatBoostRegressor(**catboost_params)
            model.fit(X_train_prep, y_train_log,
                      eval_set=[(X_val_prep, y_val_log)],
                      verbose=0 # Отключаем вывод
                     )

            # 5. Предсказание и оценка
            val_preds_log = model.predict(X_val_prep)
            # Преобразуем обратно в исходную шкалу
            val_preds_orig = np.expm1(val_preds_log)
            y_val_orig = np.expm1(y_val_log)
            # Обработка возможных отрицательных значений после expm1 (хотя маловероятно с log1p)
            val_preds_orig[val_preds_orig < 0] = 0
            fold_rmse = np.sqrt(mean_squared_error(y_val_orig, val_preds_orig))
            fold_rmses.append(fold_rmse)

            # 6. Отчет для Optuna Pruning (опционально, но полезно для долгих подборов)
            # Сообщаем Optuna промежуточный результат (RMSE на текущем фолде)
            trial.report(fold_rmse, step=fold)

            # Проверяем, нужно ли досрочно остановить текущий trial (если он явно плохой)
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        except Exception as e:
             print(f"Error during fold {fold+1} for trial {trial.number}: {e}")
             # Если фолд не удался, возвращаем очень большое значение, чтобы Optuna считала этот trial плохим
             return float('inf')

    # 7. Возвращаем средний RMSE по всем фолдам (Optuna будет это минимизировать)
    mean_rmse = np.mean(fold_rmses)
    print(f"Trial {trial.number} finished with Mean CV RMSE: {mean_rmse:.4f}")
    return mean_rmse

# --- Запуск исследования Optuna ---
# Создаем исследование, указываем направление оптимизации (minimize)
# Pruner - механизм для досрочной остановки плохих trials
study = optuna.create_study(direction='minimize',
                            pruner=optuna.pruners.MedianPruner(n_warmup_steps=2)) # Начинаем проверку на prune после 2 фолдов

# Запускаем оптимизацию, n_trials - количество попыток подбора параметров
N_TRIALS = 200 # Например, 50 или 100
study.optimize(objective, n_trials=N_TRIALS, n_jobs=1) # n_jobs=-1 может конфликтовать с CatBoost/MLflow, лучше 1

# --- Результаты ---
print("\nOptimization Finished!")
print(f"Number of finished trials: {len(study.trials)}")

best_trial = study.best_trial
print("Best trial:")
print(f"  Value (Mean CV RMSE): {best_trial.value:.4f}")
print("  Params: ")
for key, value in best_trial.params.items():
    print(f"    {key}: {value}")

# Сохраняем лучшие параметры для использования в финальном обучении
best_params = best_trial.params
# Добавляем параметры, которые не подбирались, но нужны для модели
best_params['loss_function'] = 'RMSE'
best_params['random_seed'] = RANDOM_STATE
best_params['verbose'] = 0
# Early stopping не нужен при финальном обучении с фиксированным числом итераций
# best_params['early_stopping_rounds'] = 100 # Или можно использовать best_iteration из trial? Optuna не хранит это напрямую

# Логгирование лучших параметров в MLflow (если нужно)
# try:
#     with mlflow.start_run(run_id=run.info.run_id): # Используем предыдущий run или создаем новый
#          mlflow.log_params({f"best_{k}": v for k, v in best_trial.params.items()})
#          mlflow.log_metric("best_cv_rmse_optuna", best_trial.value)
# except Exception as e:
#     print(f"Could not log best Optuna params to MLflow: {e}")

# Теперь можно использовать best_params для финального обучения модели
# (либо на всех данных с подобранным числом итераций,
# либо в цикле CV с этими параметрами для генерации сабмишна)

[I 2025-04-02 23:54:35,876] A new study created in memory with name: no-name-d25be354-9564-4677-ae16-aa17262750d5
[I 2025-04-02 23:54:40,019] Trial 0 finished with value: 26774.68633421593 and parameters: {'iterations': 2035, 'depth': 4, 'learning_rate': 0.03746270467239184, 'l2_leaf_reg': 3.979194388787327, 'bagging_temperature': 0.11655891144766806}. Best is trial 0 with value: 26774.68633421593.


Trial 0 finished with Mean CV RMSE: 26774.6863


[I 2025-04-02 23:54:47,055] Trial 1 finished with value: 26756.206261445397 and parameters: {'iterations': 1084, 'depth': 6, 'learning_rate': 0.03691551498362596, 'l2_leaf_reg': 7.135837611128692, 'bagging_temperature': 0.5165345602611249}. Best is trial 1 with value: 26756.206261445397.


Trial 1 finished with Mean CV RMSE: 26756.2063


[I 2025-04-02 23:55:10,358] Trial 2 finished with value: 26854.743496323546 and parameters: {'iterations': 2633, 'depth': 7, 'learning_rate': 0.014882085035641372, 'l2_leaf_reg': 1.0339710914195908, 'bagging_temperature': 0.18167924817042236}. Best is trial 1 with value: 26756.206261445397.


...

Trial 197 finished with Mean CV RMSE: 25730.3226


[I 2025-04-03 00:27:50,303] Trial 198 finished with value: 25857.52277850454 and parameters: {'iterations': 1853, 'depth': 5, 'learning_rate': 0.01690129941483347, 'l2_leaf_reg': 2.091004494527789, 'bagging_temperature': 0.9793406484004782}. Best is trial 191 with value: 25475.203437052394.


Trial 198 finished with Mean CV RMSE: 25857.5228


[I 2025-04-03 00:27:58,966] Trial 199 finished with value: 25769.547746629454 and parameters: {'iterations': 2475, 'depth': 5, 'learning_rate': 0.015403607085010972, 'l2_leaf_reg': 2.316953001836452, 'bagging_temperature': 0.5070450178712599}. Best is trial 191 with value: 25475.203437052394.


Trial 199 finished with Mean CV RMSE: 25769.5477

Optimization Finished!
Number of finished trials: 200
Best trial:
  Value (Mean CV RMSE): 25475.2034
  Params: 
    iterations: 2279
    depth: 5
    learning_rate: 0.01489174151229139
    l2_leaf_reg: 2.3495954457881623
    bagging_temperature: 0.9217322652060611


In [17]:
best_params

{'iterations': 2279,
 'depth': 5,
 'learning_rate': 0.01489174151229139,
 'l2_leaf_reg': 2.3495954457881623,
 'bagging_temperature': 0.9217322652060611,
 'loss_function': 'RMSE',
 'random_seed': 42,
 'verbose': 0}